In [10]:
# A = np.array([1, 3, 5, 7])
# B = np.array([2, 4, 6, 8])
# np.insert(B, np.arange(len(A)), A)
# array([1, 2, 3, 4, 5, 6, 7, 8])


import json
import numpy as np
from os import rename
import cv2
import os
from detectron2.structures import BoxMode
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

# define width and height manually
#width = 1280
#height =720
#shape = (width, height) #(2992,2992) 
lic_id = 0
date_created =""
data = {}
images=[]
annotations=[]
image_path= r"D:\Nyi Zaw Aung\815_CowDataChecking\Sumiyoshi ToAnnotate\JUne 2025\Annotate_16_June_2025"
json_name = "via_project_16Jun2025_13h10m_json"
#json_path = "G:\\Nyi Zaw Aung\\815_CowDataChecking\\Sumiyoshi ToAnnotate\\Cam60102-70304\\Yolo_Cam60102_70304_v2\\training\\"
with open(f'{image_path}\\{json_name}.json') as f1:   #D:/Projects/WH/Python/Detectron2/Yoshii_22070506/Yoshii20220705_28_000/001_220705060000_6001_20_00_28_00/via_project_8Aug2022_11h36m_json.json
    data_json = json.load(f1)
#print(len(data_json))
#print(len(data_json.values()))
Values = data_json.values()

#print(Values)

ann=0
id=0
count = 0
for a,val in enumerate(data_json.values()):      
    #mask_1 = np.zeros(shape)
    #print(val)
    #print(val)
    try:
        filename = val['filename']
    except :
        continue
    filePath= os.path.join(image_path,filename)
    #print(filePath)
    if cv2.imread(filePath) is not None:
        id += 1
        count += 1
        height,width = cv2.imread(filePath).shape[:2]
        #print(filename)

        # filename = "005_220705060000_{}.jpeg".format(count)
        # print(filename)

        # newfilename = "005_220705060000_{}.jpeg".format(count)
        # print(newfilename)

        # name = rename(filename,newfilename)
        # print(name)

        Regions = val['regions']
        images.append(
            {'id': id, 'width': width, 'height': height, 'file_name': filename, 'license': 0, 'date_captured': ''})
        for r in range(len(Regions)):
            seg = []
            
            #try:
                
            
            #classId = 1 
            #print(filePath)
            emptyDict = {}
            if val['regions'][r]['region_attributes'] == emptyDict:
                continue
                
            # try:
            #     classId = val['regions'][r]['region_attributes']['cow']
            # except:
            #     dd = val['regions'][r]['region_attributes']
                
                classId = val['regions'][r]['region_attributes']['label']
            classId = 1
            #if classId == 'cow' or classId == '0' or classId == 'undefined' :
            #    classId = 1
            #elif classId == '99':
            #    continue
            
            
        #print(f"{classId}-{count}")
            x_points = val['regions'][r]['shape_attributes']['all_points_x']
            y_points = val['regions'][r]['shape_attributes']['all_points_y']
            min_x = min(x_points)
            max_x = max(x_points)
            min_y = min(y_points)
            max_y = max(y_points)
            w = max_x - min_x
            h = max_y - min_y
            bbox = [min_x, min_y, w, h]
            area = w * h
            A = np.array(x_points)
            B = np.array(y_points)
            seg_points = np.insert(B, np.arange(len(A)), A)
            seg.append(seg_points)
            ann += 1
            annotations.append(
                {'segmentation': np.array(seg).tolist(), 'area': area, 'bbox': bbox, 'iscrowd': 0, 'ind': ann,"bbox_mode": BoxMode.XYWH_ABS,
                    'image_id': id, 'category_id': int(classId)})
            #except Exception as e:
                
                #print(filePath,' ',val['regions'][r]['region_attributes'])
                #break
# print(len(val['regions']))

data['info']={'year': 2022, 'version': '1.0', 'description': 'VIA project exported to COCO format using VGG Image Annotator (http://www.robots.ox.ac.uk/~vgg/software/via/)', 'contributor': '', 'url': 'http://www.robots.ox.ac.uk/~vgg/software/via/', 'date_created': 'Thu May 26 2022 13:08:31 GMT+0900 (Japan Standard Time)'}
data['images']=images
data['annotations']=annotations
data['licenses']=[{'id':0,'name':'Unknown License','url':''}]
#data['categories']=[{'supercategory':'cow','id':1,'name':'cow'},{'supercategory':'cow','id':2,'name':'mounting'},{'supercategory':'cow','id':3,'name':'background'}]#{'supercategory':'cow','id':98,'name':'background'}
data['categories']=[{'supercategory':'','id':1,'name':'cow'},{'supercategory':'','id':2,'name':'background'}]#{'supercategory':'cow','id':98,'name':'background'}
with open(f'{image_path}\\auto_{json_name}_COCO.json', 'w') as outfile:
   json.dump(data, outfile, cls=NumpyEncoder)
print("done")

done


In [3]:
import tkinter as tk
from tkinter import filedialog, Button, Frame, Label
from PIL import Image, ImageTk
import numpy as np
import cv2

class SimpleImageAdjuster:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Template Adjuster")
        self.root.geometry("1000x700")
        
        # Variables
        self.image_path = None
        self.template_path = None
        self.original_image = None
        self.template_image = None
        self.display_image = None
        
        # Image adjustment parameters
        self.zoom_factor = 1.0
        self.x_offset = 0
        self.y_offset = 0
        
        # Setup UI
        self.setup_ui()
        
    def setup_ui(self):
        # Main layout
        main_frame = Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Control panel
        control_panel = Frame(main_frame, bg="#f0f0f0", padx=15, pady=15)
        control_panel.pack(side=tk.TOP, fill=tk.X)
        
        # Image display area
        self.canvas_frame = Frame(main_frame, bg="#ffffff")
        self.canvas_frame.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)
        
        self.canvas = tk.Canvas(self.canvas_frame, bg="#dddddd")
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Load image buttons
        Button(control_panel, text="Load Image", command=self.load_image, 
               bg="#4CAF50", fg="white", width=15, height=1).grid(row=0, column=0, padx=5, pady=5)
        
        Button(control_panel, text="Load Template", command=self.load_template, 
               bg="#2196F3", fg="white", width=15, height=1).grid(row=0, column=1, padx=5, pady=5)
        
        # Zoom controls
        zoom_frame = Frame(control_panel, bg="#f0f0f0")
        zoom_frame.grid(row=1, column=0, columnspan=2, pady=10)
        
        Label(zoom_frame, text="Zoom:", bg="#f0f0f0").grid(row=0, column=0, padx=5)
        Button(zoom_frame, text="Zoom In (+)", command=self.zoom_in, width=10).grid(row=0, column=1, padx=5)
        Button(zoom_frame, text="Zoom Out (-)", command=self.zoom_out, width=10).grid(row=0, column=2, padx=5)
        
        # Direction controls
        direction_frame = Frame(control_panel, bg="#f0f0f0")
        direction_frame.grid(row=2, column=0, columnspan=2, pady=10)
        
        Button(direction_frame, text="↑", command=self.move_up, width=5, height=2).grid(row=0, column=1, padx=2, pady=2)
        Button(direction_frame, text="←", command=self.move_left, width=5, height=2).grid(row=1, column=0, padx=2, pady=2)
        Button(direction_frame, text="→", command=self.move_right, width=5, height=2).grid(row=1, column=2, padx=2, pady=2)
        Button(direction_frame, text="↓", command=self.move_down, width=5, height=2).grid(row=2, column=1, padx=2, pady=2)
        
        # Reset and save buttons
        Button(control_panel, text="Reset", command=self.reset_adjustments,
               bg="#FF5722", fg="white", width=10).grid(row=3, column=0, padx=5, pady=10)
        Button(control_panel, text="Save Result", command=self.save_result,
               bg="#673AB7", fg="white", width=10).grid(row=3, column=1, padx=5, pady=10)
    
    def load_image(self):
        self.image_path = filedialog.askopenfilename(
            title="Select Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")]
        )
        
        if self.image_path:
            self.original_image = cv2.imread(self.image_path)
            self.original_image = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB)
            self.reset_adjustments()
    
    def load_template(self):
        self.template_path = filedialog.askopenfilename(
            title="Select Template",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")]
        )
        
        if self.template_path:
            self.template_image = cv2.imread(self.template_path, cv2.IMREAD_UNCHANGED)
            if self.template_image.shape[2] == 3:  # No alpha channel
                self.template_image = cv2.cvtColor(self.template_image, cv2.COLOR_BGR2RGBA)
                # Add alpha channel
                alpha = np.ones(self.template_image.shape[:2], dtype=self.template_image.dtype) * 128  # Semi-transparent
                self.template_image = cv2.merge((self.template_image[:,:,:3], alpha))
            else:
                self.template_image = cv2.cvtColor(self.template_image, cv2.COLOR_BGRA2RGBA)
            
            self.update_display()
    
    # Zoom controls
    def zoom_in(self):
        self.zoom_factor += 0.1
        self.update_display()
        
    def zoom_out(self):
        if self.zoom_factor > 0.2:  # Prevent zooming out too much
            self.zoom_factor -= 0.1
            self.update_display()
    
    # Movement controls
    def move_up(self):
        self.y_offset -= 20
        self.update_display()
        
    def move_down(self):
        self.y_offset += 20
        self.update_display()
        
    def move_left(self):
        self.x_offset -= 20
        self.update_display()
        
    def move_right(self):
        self.x_offset += 20
        self.update_display()
    
    def reset_adjustments(self):
        if self.original_image is not None:
            self.zoom_factor = 1.0
            self.x_offset = 0
            self.y_offset = 0
            self.update_display()
    
    def update_display(self):
        if self.original_image is None:
            return
            
        # Get canvas dimensions
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        if canvas_width <= 1 or canvas_height <= 1:
            # Canvas not ready yet
            self.root.after(100, self.update_display)
            return
        
        # Calculate new image dimensions based on zoom
        img_height, img_width = self.original_image.shape[:2]
        new_width = int(img_width * self.zoom_factor)
        new_height = int(img_height * self.zoom_factor)
        
        # Resize image
        resized_img = cv2.resize(self.original_image, (new_width, new_height), 
                                interpolation=cv2.INTER_AREA if self.zoom_factor < 1 else cv2.INTER_LINEAR)
        
        # Create a blank canvas the size of the display area
        canvas_image = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
        canvas_image.fill(255)  # White background
        
        # Calculate position to paste the image
        center_x = canvas_width // 2 + self.x_offset
        center_y = canvas_height // 2 + self.y_offset
        
        x1 = max(0, int(center_x - new_width // 2))
        y1 = max(0, int(center_y - new_height // 2))
        x2 = min(canvas_width, x1 + new_width)
        y2 = min(canvas_height, y1 + new_height)
        
        # Calculate the portion of the resized image to paste
        img_x1 = max(0, -int(center_x - new_width // 2))
        img_y1 = max(0, -int(center_y - new_height // 2))
        img_x2 = img_x1 + (x2 - x1)
        img_y2 = img_y1 + (y2 - y1)
        
        # Paste the visible portion of the image
        if x1 < x2 and y1 < y2 and img_x1 < new_width and img_y1 < new_height:
            canvas_image[y1:y2, x1:x2] = resized_img[img_y1:img_y2, img_x1:img_x2]
        
        # Overlay template if available
        current_image = canvas_image.copy()
        if self.template_image is not None:
            # Resize template to match canvas
            template_resized = cv2.resize(self.template_image, (canvas_width, canvas_height))
            
            # Extract RGB and alpha channels
            template_rgb = template_resized[:, :, :3]
            template_alpha = template_resized[:, :, 3] / 255.0 * 0.5  # Fixed 50% opacity
            
            # Create an alpha mask for blending
            alpha_mask = np.stack([template_alpha] * 3, axis=2)
            
            # Blend the template with the current image
            current_image = (1 - alpha_mask) * canvas_image + alpha_mask * template_rgb
            current_image = current_image.astype(np.uint8)
        
        # Convert to TkInter format and display
        self.display_image = ImageTk.PhotoImage(image=Image.fromarray(current_image))
        
        # Clear and update canvas
        self.canvas.delete("all")
        self.canvas.create_image(0, 0, image=self.display_image, anchor=tk.NW)
        
        # Display current zoom and position information
        status_text = f"Zoom: {self.zoom_factor:.1f}x | Position: X={self.x_offset}, Y={self.y_offset}"
        self.canvas.create_text(10, 10, text=status_text, anchor=tk.NW, fill="blue")
    
    def save_result(self):
        if self.original_image is not None:
            save_path = filedialog.asksaveasfilename(
                defaultextension=".png",
                filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg"), ("All files", "*.*")]
            )
            
            if save_path:
                # Get the current displayed image
                canvas_width = self.canvas.winfo_width()
                canvas_height = self.canvas.winfo_height()
                
                # Create a version without the template for saving
                img_height, img_width = self.original_image.shape[:2]
                new_width = int(img_width * self.zoom_factor)
                new_height = int(img_height * self.zoom_factor)
                
                # Resize image
                resized_img = cv2.resize(self.original_image, (new_width, new_height), 
                                        interpolation=cv2.INTER_AREA if self.zoom_factor < 1 else cv2.INTER_LINEAR)
                
                # Create a blank canvas the size of the display area
                final_image = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
                final_image.fill(255)  # White background
                
                # Calculate position to paste the image
                center_x = canvas_width // 2 + self.x_offset
                center_y = canvas_height // 2 + self.y_offset
                
                x1 = max(0, int(center_x - new_width // 2))
                y1 = max(0, int(center_y - new_height // 2))
                x2 = min(canvas_width, x1 + new_width)
                y2 = min(canvas_height, y1 + new_height)
                
                # Calculate the portion of the resized image to paste
                img_x1 = max(0, -int(center_x - new_width // 2))
                img_y1 = max(0, -int(center_y - new_height // 2))
                img_x2 = img_x1 + (x2 - x1)
                img_y2 = img_y1 + (y2 - y1)
                
                # Paste the visible portion of the image
                if x1 < x2 and y1 < y2 and img_x1 < new_width and img_y1 < new_height:
                    final_image[y1:y2, x1:x2] = resized_img[img_y1:img_y2, img_x1:img_x2]
                
                # Save the image
                cv2.imwrite(save_path, cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR))
                tk.messagebox.showinfo("Success", f"Image saved to {save_path}")

def main():
    root = tk.Tk()
    app = SimpleImageAdjuster(root)
    root.mainloop()

if __name__ == "__main__":
    main()